# Pesquisando por notícias que divulguem iniciativas/projetos com Dados Abertos Governamentais

## Palavras de busca

In [1]:
first_search_strings = [
    'dados abertos governamentais',
    'dados governamentais',
    'dados públicos abertos',
    'dados do governo',
    'análise de dados do governo',
    'portal de dados do governo',
    'portal de dados governamentais',
    'portal público do governo',
    'portal de dados abertos do governo',
    'publicação de dados abertos do governo',
    'publicação de dados governamentais',
    'consumir dados abertos do governo',
    'consumir dados abertos governamentais',
]

Após algumas interações com essas strings, percebeu-se que elas traziam muitos resultados que abordavam atividades e acontecimentos que envolviam dados e o governo porém não falava de projetos em si. A paritr disso novas strings foram testadas e trouxeram resultados melhores.

In [2]:
search_strings = [
    '"projeto" and "dados governo"',
    '"projeto" and "dados governamentais"',
    '"projeto" and "monitora" and "dados" and "governo"'
]

### Via NEWSAPI

In [3]:
import newsapi as na
import requests

In [4]:
def get_status(search):
    print("Status: {0}\nTotal results: {1}".format(all_articles.get('status'), 
                                                   all_articles.get('totalResults')))


In [5]:
def print_news(results):
    
    for result in results.get('articles'):
        print('- ' + result.get('title'))

In [6]:
na.const.languages

{'ar',
 'cn',
 'de',
 'en',
 'es',
 'fr',
 'he',
 'it',
 'nl',
 'no',
 'pt',
 'ru',
 'sv',
 'ud'}

In [7]:
newsapi = na.NewsApiClient(api_key='S')

In [46]:
top_headlines = newsapi.get_top_headlines(q='dados')
top_headlines

{'status': 'ok', 'totalResults': 0, 'articles': []}

In [102]:
all_articles = newsapi.get_everything(q=first_search_strings[0],
                                    language='pt')

get_status(all_articles)

Status: ok
Total results: 3


In [103]:
print_news(all_articles)

- Especialista fala sobre pontos polêmicos do projeto de lei do streaming no Brasi
- O que mudou, ou sobrou, na Funai após 100 dias de gestão ruralista? Por Oswaldo Braga de Souza
- [Akitando] #47 - Entendendo "Devops" para Iniciantes em Programação (Parte 1) | Série "Começando aos 40"


### Extração API(RSS) do Google News

In [104]:
from xml.etree import ElementTree
import pandas as pd
import time

In [105]:
def print_news_rss(results):
    
    for result in results:
        print('    ')
        print("Titulo: " + result.findtext('title'))
        print("Data da Publicação: " + result.findtext('pubDate'))
        print("Link: " + result.findtext('link'))
        print("Canal: " + result.findtext('source'))

In [106]:
def add_result(result):
    df = pd.DataFrame([[
                        result.findtext('title'),
                        result.findtext('pubDate'),
                        result.findtext('link'),
                        result.findtext('source'),
                        str(time.time()).split('.')[0]]], columns=columns)

    return df

In [107]:
def extract_results(results, results_df):
    
    for result in results:
        results_df = pd.concat([results_df, add_result(result)], ignore_index=True, sort=False)
        
    return results_df

In [108]:
columns=['title',
         'pubDate',
         'link',
         'source', 
         'timestamp_extract']

In [120]:
results_df = pd.DataFrame(columns=['title',
                                        'pubDate',
                                        'link',
                                        'source',
                                        'timestamp_extract',
                                  ])

In [110]:
url = 'https://news.google.com/rss/search?q='

In [114]:
aux = '&hl=pt-BR&gl=BR&ceid=BR:pt-419'

In [128]:
def search_rss(string, results_df):
    
    print('Extraindo notícias para a string {0}'.format(string))
    url_final = url + string + aux
    response = requests.get(url_final)
    
    if response.status_code == 200:
        
        tree = ElementTree.fromstring(response.content)
        tree = tree.find('channel')
        news = tree.getchildren()
        news = news[8:] # As 8 primeiras posições são metadados
        
        print('Foram encontrados {0} resultados'.format(len(news)))
        
        results_df = extract_results(noticias, results_df)
    
    print('--')
    
    return results_df

In [129]:
for string in search_strings:
    
    results_df = search_rss(string, results_df)

Extraindo notícias para a string "projeto" and "dados governo"
Foram encontrados 10 resultados
--
Extraindo notícias para a string "projeto" and "dados governamentais"
Foram encontrados 69 resultados
--
Extraindo notícias para a string "projeto" and "monitora" and "dados" and "governo"
Foram encontrados 100 resultados
--


In [134]:
results_df.head()

,title,pubDate,link,source,timestamp_extract
0,Governo cria base de dados com informações sob...,"Thu, 10 Oct 2019 07:00:00 GMT",https://www.gazetadopovo.com.br/republica/brev...,Gazeta do Povo,1572983744
1,Projeto recupera decreto sobre comunicação de ...,"Fri, 14 Jun 2019 07:00:00 GMT",https://portal.comunique-se.com.br/projeto-rec...,Portal Comunique-se,1572983744
2,Cinturão Digital do Ceará viabiliza qualidade ...,"Tue, 20 Aug 2019 07:00:00 GMT",https://www.ceara.gov.br/2019/08/20/cinturao-d...,Ceará,1572983744
3,Bolsonaro sanciona lei do cadastro positivo pa...,"Mon, 08 Apr 2019 07:00:00 GMT",https://economia.estadao.com.br/noticias/geral...,Estadão,1572983744
4,"Governo veta ABDI no Ministério da Ciência, Te...","Wed, 19 Jun 2019 07:00:00 GMT",http://www.telesintese.com.br/governo-veta-abd...,Telesintese.,1572983744


In [135]:
results_df.describe()

,title,pubDate,link,source,timestamp_extract
count,30,30,30,30,30
unique,10,10,10,9,3
top,Cinturão Digital do Ceará viabiliza qualidade ...,"Wed, 30 May 2018 07:00:00 GMT",https://www.ceara.gov.br/2018/04/04/secretaria...,Ceará,1572983745
freq,3,3,3,6,10
